In [273]:
library('DBI')
library('RPostgreSQL')
library('dbplyr')
library('dplyr')
library(tidyr)

# connect to database
con <- dbConnect(PostgreSQL(), host="localhost", port=5432, dbname="knock100", user="guest", password="guest")

# get access to table
receipt_tbl <-  tbl(con,'receipt')
customer_tbl <- tbl(con, 'customer')

### R-041
レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [276]:
receipt_tbl %>%
group_by(sales_ymd) %>%
summarise(sales_today = sum(amount)) %>%
mutate(sales_yest = sales_today %>% lag()) %>%
mutate(diff = sales_today - sales_yest) %>%
head

# Source:   lazy query [?? x 4]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  sales_ymd sales_today sales_yest   diff
      <int>       <dbl>      <dbl>  <dbl>
1  20190504       49561         NA     NA
2  20180414       27599      49561 -21962
3  20170416       16442      27599 -11157
4  20190422       37044      16442  20602
5  20190710       27796      37044  -9248
6  20181027       28497      27796    701

In [3]:
re %>%
group_by(sales_ymd) %>%
summarise(sales_today = sum(amount)) %>%
mutate(sales_yest = ifelse(is.na(lag(sales_today)),0,lag(sales_today))) %>%
mutate(diff = sales_today - sales_yest) %>%
head

Warning message:
“Missing values are always removed in SQL.
Use `SUM(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”


# Source:   lazy query [?? x 4]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  sales_ymd sales_today sales_yest   diff
      <int>       <dbl>      <dbl>  <dbl>
1  20190504       49561          0  49561
2  20180414       27599      49561 -21962
3  20170416       16442      27599 -11157
4  20190422       37044      16442  20602
5  20190710       27796      37044  -9248
6  20181027       28497      27796    701

### R-042
レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [4]:
receipt_tbl %>%
group_by(sales_ymd) %>%
summarise(sales_today = sum(amount)) %>%
mutate(sales_1 = ifelse(is.na(lag(sales_today,n=1)),0,lag(sales_today,n=1)),
		sales_2 = ifelse(is.na(lag(sales_today,n=2)),0,lag(sales_today,n=2)),
		sales_3 = ifelse(is.na(lag(sales_today,n=3)),0,lag(sales_today,n=3))
) %>%
head

# Source:   lazy query [?? x 5]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  sales_ymd sales_today sales_1 sales_2 sales_3
      <int>       <dbl>   <dbl>   <dbl>   <dbl>
1  20190504       49561       0       0       0
2  20180414       27599   49561       0       0
3  20170416       16442   27599   49561       0
4  20190422       37044   16442   27599   49561
5  20190710       27796   37044   16442   27599
6  20181027       28497   27796   37044   16442

In [305]:
library(stringr)

D <- 3

# lag formulas to apply
fms <- list()
for (d in 1:D) fms[[d]] <- stringr::str_c('~ lag(.x, n = ', d, ')') %>% as.formula()

receipt_tbl %>%
  group_by(sales_ymd) %>%
  summarise(sales_daily= sum(sales_ymd)) %>%
  collect() %>%
  mutate(
		across(
			.cols = sales_daily,
			.fns = fms,
			.names = '{.col}_lag_{.fn}'
    )
  ) %>% head

sales_ymd,sales_daily,sales_daily_lag_1,sales_daily_lag_2,sales_daily_lag_3
<int>,<dbl>,<dbl>,<dbl>,<dbl>
20190504,2382479472,NA,NA,NA
20180414,2118943470,2382479472,NA,NA
20170416,1593462864,2118943470,2382479472,NA
20190422,2261327264,1593462864,2118943470,2382479472
20190710,2059452420,2261327264,1593462864,2118943470
20181027,1836473457,2059452420,2261327264,1593462864


In [221]:
library(stringr)
# number of lags
D <- 3

sales_lag <- receipt_tbl %>%
  group_by(sales_ymd) %>%
  summarise(sales_daily= sum(sales_ymd))
  
for(d in 1:D){
	sales_lag %>% mutate(
				across(
      .cols = sales_lag %>% ncol(),
      .fns = ~ lag(.x, n = 1),
	  .names = str_c('{.col}',d)
    )
  ) -> sales_lag
}
sales_lag %>% rename_with(.fn = function(x){str_sub(x,start = 1, end = 11) %>% str_c('_lag_', str_sub(x, start=-1,end=-1))}, .cols = -(D-1):-1)%>% head

# Source:   lazy query [?? x 5]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  sales_ymd sales_daily sales_daily_lag_1 sales_daily_lag_2 sales_daily_lag_3
      <int>       <dbl>             <dbl>             <dbl>             <dbl>
1  20190504  2382479472                NA                NA                NA
2  20180414  2118943470        2382479472                NA                NA
3  20170416  1593462864        2118943470        2382479472                NA
4  20190422  2261327264        1593462864        2118943470        2382479472
5  20190710  2059452420        2261327264        1593462864        2118943470
6  20181027  1836473457        2059452420        2261327264        1593462864

In [329]:
library(stringr)

receipt_tbl %>%
    group_by(sales_ymd) %>%
    summarise(sales_daily = sum(sales_ymd)) %>%
    mutate(
        across(
            .cols = sales_daily,
            .fns = list(
				lag_1 = ~ lag(.x, n = 1),
			 	lag_2 = ~ lag(.x, n = 2),
				lag_3 = ~ lag(.x, n = 3)
			),
        )
    ) %>% head()

# Source:   lazy query [?? x 5]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  sales_ymd sales_daily sales_daily_lag_1 sales_daily_lag_2 sales_daily_lag_3
      <int>       <dbl>             <dbl>             <dbl>             <dbl>
1  20190504  2382479472                NA                NA                NA
2  20180414  2118943470        2382479472                NA                NA
3  20170416  1593462864        2118943470        2382479472                NA
4  20190422  2261327264        1593462864        2118943470        2382479472
5  20190710  2059452420        2261327264        1593462864        2118943470
6  20181027  1836473457        2059452420        2261327264        1593462864

### R-043:
レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。

ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [81]:
cu = tbl(con,'customer')
re %>%
inner_join(cu, by = 'customer_id') %>%
mutate(generation = age - age%%10) %>%
group_by(generation, gender_cd) %>%
summarise(sales_amount = amount %>% sum(), .groups = 'drop') %>% 
pivot_wider(names_from = gender_cd, names_prefix = 'sales', values_from = sales_amount, values_fill = 0) %>%
rename(sales_male = sales0, sales_female = sales1, sales_other = sales9) -> sales_summary
sales_summary 

# Source:   lazy query [?? x 4]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  generation sales_male sales_female sales_other
       <dbl>      <dbl>        <dbl>       <dbl>
1         10       1591       149836        4317
2         20      72940      1363724       44328
3         30     177322       693047       50441
4         40      19355      9320791      483512
5         50      54320      6685192      342923
6         60     272469       987741       71418
7         70      13435        29764        2427
8         80      46360       262923        5111
9         90          0         6260           0

### R-044
043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする

In [85]:
sales_summary %>%
rename('00'=sales_male, '01'=sales_female, '99'=sales_other) %>%
  pivot_longer(cols = - generation, names_to = 'gender_cd', values_to = 'sales_amount') %>% head

# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  generation gender_cd sales_amount
       <dbl> <chr>            <dbl>
1         10 00                1591
2         20 00               72940
3         30 00              177322
4         40 00               19355
5         50 00               54320
6         60 00              272469

In [6]:
sales_summary %>%
select(
	generation,
	'01' = sales_female,
	'00' = sales_male,
	'99' = sales_unknown
	) %>%
pivot_longer(!generation, names_to = 'gender', values_to = 'sales') %>%
head(10)

# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
   generation gender   sales
        <dbl> <chr>    <dbl>
 1         10 01      149836
 2         20 01     1363724
 3         30 01      693047
 4         40 01     9320791
 5         50 01     6685192
 6         60 01      987741
 7         70 01       29764
 8         80 01      262923
 9         90 01        6260
10         10 00        1591

### R-045
顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [48]:
cu %>%
transmute(customer_id ,date = birth_day %>% as.character() %>% str_replace_all(pattern = '-',replacement='')) %>%
head

# Source:   lazy query [?? x 2]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    date    
  <chr>          <chr>   
1 CS021313000114 19810429
2 CS037613000071 19520401
3 CS031415000172 19761004
4 CS028811000001 19330327
5 CS001215000145 19950329
6 CS020401000016 19740915

### R-046
顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [8]:
customer_tbl %>%
transmute(customer_id, application_date = as.Date(application_date)) %>%
head

# Source:   lazy query [?? x 2]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    application_date
  <chr>          <date>          
1 CS021313000114 2015-09-05      
2 CS037613000071 2015-04-14      
3 CS031415000172 2015-05-29      
4 CS028811000001 2016-01-15      
5 CS001215000145 2017-06-05      
6 CS020401000016 2015-02-25      

### R-047
レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [47]:
re %>%
transmute(sales_date = sales_ymd %>% as.character() %>% as.Date(),
			receipt_no,receipt_sub_no) %>%
head

# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  sales_date receipt_no receipt_sub_no
  <date>          <int>          <int>
1 2018-11-03        112              1
2 2018-11-18       1132              2
3 2017-07-12       1102              1
4 2019-02-05       1132              1
5 2018-08-21       1102              2
6 2019-06-05       1112              1

### R-052:
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [6]:
library(stringr)
re %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
mutate(amount_over_2000 = if_else(amount>2000,1,0)) %>%
head(10)

# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
   customer_id    amount amount_over_2000
   <chr>           <dbl>            <dbl>
 1 CS001311000059   2302                1
 2 CS004614000122    248                0
 3 CS003512000043    298                0
 4 CS011615000061    246                0
 5 CS029212000033   3604                1
 6 CS007515000119   7157                1
 7 CS034515000123   3699                1
 8 CS004315000058    490                0
 9 CS026414000014   6671                1
10 CS001615000099    768                0

### R-053:
顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [86]:
cu = tbl(con,'customer')

cu %>% mutate(is_tokyo = dplyr::between(substr(postal_cd,1,3) %>% as.integer(),100,209) %>% ifelse(1,0)) %>%
inner_join(re, by = 'customer_id') %>%
group_by(is_tokyo) %>%
summarise(amount=n_distinct(customer_id)) %>% head

# Source:   lazy query [?? x 2]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  is_tokyo amount
     <dbl>  <dbl>
1        0   3906
2        1   4400

### R-054:
顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [12]:
cu %>%
mutate(prefecture_cd =
		case_when(
			substr(address,1,2) == '埼玉' ~ '11',
			substr(address,1,2) == '千葉' ~ '12',
			substr(address,1,2) == '東京' ~ '13',
			substr(address,1,3) == '神奈川' ~ '14'
		)) %>%
select(customer_id, address, prefecture_cd) %>%
head

# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    address                            prefecture_cd
  <chr>          <chr>                              <chr>        
1 CS021313000114 神奈川県伊勢原市粟窪**********     14           
2 CS037613000071 東京都江東区南砂**********         13           
3 CS031415000172 東京都渋谷区代々木**********       13           
4 CS028811000001 神奈川県横浜市泉区和泉町********** 14           
5 CS001215000145 東京都大田区仲六郷**********       13           
6 CS020401000016 東京都板橋区若木**********         13           

In [21]:
pref_vec <- c("埼玉県", "千葉県", "東京都", "神奈川")
pref_map <- function(str) which(str == pref_vec) + 10 %>% as.integer()

customer_tbl <- tbl(con, 'customer')

customer_tbl %>%
    select(customer_id, address) %>%
    collect() %>%
    mutate(
        across(
            .cols = address,
            .fns = function(cls) purrr::map_int(cls, ~ substr(.x, 1, 3) %>% pref_map),
			.names = 'pref_code'
        )
    ) %>%
    head()

customer_id,address,pref_code
<chr>,<chr>,<int>
CS021313000114,神奈川県伊勢原市粟窪**********,14
CS037613000071,東京都江東区南砂**********,13
CS031415000172,東京都渋谷区代々木**********,13
CS028811000001,神奈川県横浜市泉区和泉町**********,14
CS001215000145,東京都大田区仲六郷**********,13
CS020401000016,東京都板橋区若木**********,13


### R-055:
レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
- 最小値以上第1四分位未満 ・・・ 1を付与
- 第1四分位以上第2四分位未満 ・・・ 2を付与
- 第2四分位以上第3四分位未満 ・・・ 3を付与
- 第3四分位以上 ・・・ 4を付与

In [24]:
receipt_tbl %>% 
group_by(customer_id) %>%
summarise(amount = sum(amount)) %>%
collect() %>%
mutate(
	across(
		.cols = amount,
		.fns = function(cls){
			q <- quantile(cls, probs = c(0.25, 0.5, 0.75, 1.0))
			purrr::map_int(cls, ~ which(.x <= q) %>% min())
		},
		.names = 'pct_group'
)) %>% head

customer_id,amount,pct_group
<chr>,<dbl>,<int>
CS001311000059,2302,3
CS004614000122,248,1
CS003512000043,298,1
CS011615000061,246,1
CS029212000033,3604,3
CS007515000119,7157,4


### R-056:
顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [90]:
cu %>%
mutate(generation = ifelse(age>=60, 60, age-age%%10)) -> customer_generation

customer_generation %>% 
select(customer_id, birth_day, generation) %>%
head

# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    birth_day  generation
  <chr>          <date>          <dbl>
1 CS021313000114 1981-04-29         30
2 CS037613000071 1952-04-01         60
3 CS031415000172 1976-10-04         40
4 CS028811000001 1933-03-27         60
5 CS001215000145 1995-03-29         20
6 CS020401000016 1974-09-15         40

### R-057:
056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [91]:
generation %>%
mutate(gender_generation = gender_cd + generation) %>%
select(customer_id, gender_cd, gender, age, generation, gender_generation) %>%
head

# Source:   lazy query [?? x 6]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    gender_cd gender   age generation gender_generation
  <chr>              <int> <chr>  <int>      <dbl>             <dbl>
1 CS021313000114         1 女性      37         30                31
2 CS037613000071         9 不明      66         60                69
3 CS031415000172         1 女性      42         40                41
4 CS028811000001         1 女性      86         60                61
5 CS001215000145         1 女性      24         20                21
6 CS020401000016         0 男性      44         40                40

### R-058:
顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [16]:
cul = DBI::dbReadTable(con, "customer")

library(recipes)
library(tidymodels)

cul %>%
  transmute(customer_id, gender_cd = factor(gender_cd)) %>%
  recipe() %>%
  step_dummy(gender_cd, one_hot = T) %>%
  prep() %>%
  juice() %>%
  head(10)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”

Attaching package: ‘recipes’


The following object is masked from ‘package:stringr’:

    fixed


The following object is masked from ‘package:stats’:

    step


── Attaching packages ────────────────────────────────────── tidymodels 0.2.0 ──

✔ broom        0.8.0     ✔ rsample      0.1.1
✔ dials        0.1.1     ✔ tibble       3.1.7
✔ ggplot2      3.3.6     ✔ tune         0.2.0
✔ infer        1.0.0     ✔ workflows    0.2.6
✔ modeldata    0.1.1     ✔ workflowsets 0.2.1
✔ parsnip      0.2.1     ✔ yardstick    0.0.9
✔ purrr        0.3.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ recipes::fixed() masks stringr::fixed()
✖ dplyr::ident()   masks dbplyr::ident()
✖ dplyr::lag()     masks stats::lag()
✖ dplyr::sql()     masks dbplyr::sql()
✖ recipes::step()  mask

customer_id,gender_cd_X0,gender_cd_X1,gender_cd_X9
<fct>,<dbl>,<dbl>,<dbl>
CS021313000114,0,1,0
CS037613000071,0,0,1
CS031415000172,0,1,0
CS028811000001,0,1,0
CS001215000145,0,1,0
CS020401000016,1,0,0
CS015414000103,0,1,0
CS029403000008,1,0,0
CS015804000004,1,0,0


In [17]:
cu %>% 
transmute(customer_id, gender_cd = 
		case_when(
			gender_cd == 1~'female',
			gender_cd == 0~'male',
			gender_cd == 9~'unknown'
		)) %>%
mutate(name = gender_cd, value = 1L) %>%
pivot_wider(values_fill = 0L) %>%
head(10)

# Source:   lazy query [?? x 5]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
   customer_id    gender_cd female unknown  male
   <chr>          <chr>      <int>   <int> <int>
 1 CS001105000001 male           0       0     1
 2 CS001112000009 female         1       0     0
 3 CS001112000019 female         1       0     0
 4 CS001112000021 female         1       0     0
 5 CS001112000023 female         1       0     0
 6 CS001112000024 female         1       0     0
 7 CS001112000029 female         1       0     0
 8 CS001112000030 female         1       0     0
 9 CS001113000004 female         1       0     0
10 CS001113000010 female         1       0     0

In [95]:
cu %>% select(customer_id, gender_cd) %>%
mutate(value = 1) %>%
pivot_wider(names_from = gender_cd, names_prefix = 'gender_cd' ,values_from = value, values_fill = 0) 

# Source:   lazy query [?? x 4]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
   customer_id    gender_cd0 gender_cd9 gender_cd1
   <chr>               <dbl>      <dbl>      <dbl>
 1 CS001105000001          1          0          0
 2 CS001112000009          0          0          1
 3 CS001112000019          0          0          1
 4 CS001112000021          0          0          1
 5 CS001112000023          0          0          1
 6 CS001112000024          0          0          1
 7 CS001112000029          0          0          1
 8 CS001112000030          0          0          1
 9 CS001113000004          0          0          1
10 CS001113000010          0          0          1
# … with more rows

### R-059:
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [13]:
library(stringr)
receipt_tbl <- tbl(con, 'receipt')
re %>%
filter(!str_detect(customer_id,'^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
collect() %>%
mutate(normized_amount = amount) %>%
mutate(
	across(
		.cols = normized_amount,
		.fns = ~ scale(.x, center = TRUE, scale = TRUE)
	)) %>% head

customer_id,amount,normized_amount
<chr>,<dbl>,"<dbl[,1]>"
CS001311000059,2302,-0.09032403
CS004614000122,248,-0.84528400
CS003512000043,298,-0.82690620
CS011615000061,246,-0.84601911
CS029212000033,3604,0.38823385
CS007515000119,7157,1.69416023


In [18]:
library(stringr)
re %>%
filter(!str_detect(customer_id,'^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
mutate(std_amount = (amount - mean(amount))/sd(amount)) %>%
head

Warning message:
“Missing values are always removed in SQL.
Use `AVG(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”
Warning message:
“Missing values are always removed in SQL.
Use `STDDEV_SAMP(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”


# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    amount std_amount
  <chr>           <dbl>      <dbl>
1 CS001311000059   2302    -0.0903
2 CS004614000122    248    -0.845 
3 CS003512000043    298    -0.827 
4 CS011615000061    246    -0.846 
5 CS029212000033   3604     0.388 
6 CS007515000119   7157     1.69  

In [15]:
receipt_tbl %>%
filter(!str_detect(customer_id,'^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
collect() %>%
mutate(std_amount = scale(amount, center = TRUE, scale = TRUE)) %>%
head

customer_id,amount,std_amount
<chr>,<dbl>,"<dbl[,1]>"
CS001311000059,2302,-0.09032403
CS004614000122,248,-0.84528400
CS003512000043,298,-0.82690620
CS011615000061,246,-0.84601911
CS029212000033,3604,0.38823385
CS007515000119,7157,1.69416023


### R-060:
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [19]:
library(stringr)
re %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE )) %>%
mutate(relative_pos = as.numeric((amount - min(amount)))/(max(amount)-min(amount))) %>%
head(10)

Warning message:
“Missing values are always removed in SQL.
Use `MIN(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”
Warning message:
“Missing values are always removed in SQL.
Use `MAX(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”


# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
   customer_id    amount relative_pos
   <chr>           <dbl>        <dbl>
 1 CS001311000059   2302      0.0970 
 2 CS004614000122    248      0.00773
 3 CS003512000043    298      0.00991
 4 CS011615000061    246      0.00765
 5 CS029212000033   3604      0.154  
 6 CS007515000119   7157      0.308  
 7 CS034515000123   3699      0.158  
 8 CS004315000058    490      0.0182 
 9 CS026414000014   6671      0.287  
10 CS001615000099    768      0.0303 